In [1]:
%load_ext dotenv
%dotenv



In [2]:
%%capture

%pip install langchain langchain-community
%pip install langchainhub
%pip install langchain-chroma
%pip install langchain-groq
%pip install langchain-huggingface
%pip install gradio
%pip install langchain openai


In [3]:
# LangChain
from langchain_experimental.pal_chain.base import PALChain

from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langchain.chains import SequentialChain

import os

In [4]:
from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain


In [5]:


llm = ChatOpenAI(model_name='gpt-3.5-turbo')

/var/folders/cn/1py0zvm95nx9kp_1_zqwycrm0000gn/T/ipykernel_12917/3838871374.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-3.5-turbo')


In [6]:
SPOTIPY_PROMPT_TEMPLATE = (
    '''
API LIMITATIONS TO NOTE
* When requesting track information, the limit is 50 at a time
* When requesting audio features, the limit is 100 at a time
* When selecting multiple artists, the limit is 50 at a time
* When asking for recommendations, the limit is 100 at a time
=====

Q: What albums has the band Pink Floyd made?

# solution in Python:


def solution():
    """What albums has the band Pink Floyd made?"""
    search_results = sp.search(q='Pink Floyd', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(uri, album_type='album')
    return albums



Q: Who are some musicians similar to Kendrick Lamar?

# solution in Python:


def solution():
    """Who are some musicians similar to Kendrick Lamar?"""
    search_results = sp.search(q='Kendrick Lamar', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    artists = sp.artist_related_artists(uri)
    return artists



Q: Tell me what songs by Seu Jorge sound like

# solution in Python:


def solution():
    """Tell me what songs by Seu Jorge sound like?"""
    search_results = sp.search(q='Seu Jorge', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    tracks = sp.artist_top_tracks(uri)
    track_uris = [track.get('uri') for track in tracks['tracks']]
    audio_details = sp.audio_features(track_uris)
    return audio_details



Q: Get me the URI for the album Electric Ladyland by Jimi Hendrix

# solution in Python:


def solution():
    """Get me the URI for the album Electric Ladyland by Jimi Hendrix"""
    search_results = sp.search(q='Electric Ladyland', type='album')
    uri = search_results['albums']['items'][0].get('uri')
    return uri



Q: What are the first three songs on Led Zeppelin's IV?

# solution in Python:


def solution():
    """What are the first three songs on Led Zeppelin's IV?"""
    # Get the URI for the album
    search_results = sp.search(q='Led Zeppelin IV', type='album')
    album = search_results['albums']['items'][0]
    album_uri = album['uri']
    # Get the album tracks
    album_tracks = sp.album_tracks(album_uri)['items']
    # Sort the tracks by duration
    first_three = album_tracks[:3]
    tracks = []
    # Only include relevant fields
    for i, track in enumerate(first_three):
        tracks.append({{
            'position': i+1,
            'song_name': track.get('name'),
            'song_uri': track['artists'][0].get('uri'),
            'artist_uri': track['artists'][0].get('uri'),
            'album_uri': album.get('uri'),
            'album_name': album.get('name')
        }})
    return tracks


Q: What are the thirty most danceable songs by Pink Floyd?

# solution in Python:


def solution():
    """What are the thirty most danceable songs by Pink Floyd?"""
    search_results = sp.search(q='Pink Floyd', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(uri, album_type='album')
    album_uris = [album['uri'] for album in albums['items']]
    tracks = []
    for album_uri in album_uris:
        album_tracks = sp.album_tracks(album_uri)
        tracks.extend(album_tracks['items'])
    track_uris = [track['uri'] for track in tracks]
    danceable_tracks = []
    # You can only have 100 at a time
    for i in range(0, len(track_uris), 100):
        subset_track_uris = track_uris[i:i+100]
        audio_details = sp.audio_features(subset_track_uris)
        for j, details in enumerate(audio_details):
            if details['danceability'] > 0.7:
                track = tracks[i+j]
                danceable_tracks.append({{
                    'song': track.get('name')
                    'album': track.get('album').get('name')
                    'danceability': details.get('danceability'),
                    'tempo': details.get('tempo'),
                }})
                # Be sure to add the audio details to the track
                danceable_tracks.append(track)
    return danceable_tracks



Q: {question}. Return a list or dictionary, only including the fields necessary to answer the question, including relevant scores and the uris to the albums/songs/artists mentioned. Only return the data – if the prompt asks for a format such as markdown or a simple string, ignore it: you are only meant to provide the information, not the formatting. A later step in the process will convert the data into the new format (table, sentence, etc).

# solution in Python:
'''.strip()
    + "\n\n\n"
)

SPOTIPY_PROMPT = PromptTemplate(input_variables=["question"], template=SPOTIPY_PROMPT_TEMPLATE)


In [7]:
auth = SpotifyClientCredentials(
    client_id=os.environ['SPOTIPY_CLIENT_ID'],
    client_secret=os.environ['SPOTIPY_CLIENT_SECRET']
)
sp = spotipy.Spotify(auth_manager=auth)

In [8]:
spotify_chain = LLMChain(
    llm=llm,
    prompt=SPOTIPY_PROMPT,
    output_key="code"
)


/var/folders/cn/1py0zvm95nx9kp_1_zqwycrm0000gn/T/ipykernel_12917/4229535114.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  spotify_chain = LLMChain(


In [9]:
EXECUTE_CODE_PROMPT = PromptTemplate(
    input_variables=["code"],
    template="""
    import json
    sp = globals()['sp']
    
    {code}
    
    result = solution()
    print(json.dumps(result))
    """
)
execute_chain = LLMChain(
    llm=ChatOpenAI(temperature=0),  # Use a deterministic model for code execution
    prompt=EXECUTE_CODE_PROMPT,
    output_key="result"
)


In [10]:
RESPONSE_CLEANUP_PROMPT_TEMPLATE = (""" 
Using this code:

```python
{code}
```

We got the following output from the Spotify API:

```json
{result}
```

Using the output above as your data source, answer the question {question}. Don't describe the code or process, just answer the question.
Answer:"""
)

RESPONSE_CLEANUP_PROMPT = PromptTemplate(
    input_variables=["question", "intermediate_steps", "result"],
    template=RESPONSE_CLEANUP_PROMPT_TEMPLATE,
)

In [11]:
explainer_chain = LLMChain(
    llm=llm,
    prompt=RESPONSE_CLEANUP_PROMPT,
    verbose=True,
    output_key='answer'
)

In [12]:
overall_chain = SequentialChain(
    chains=[spotify_chain, execute_chain, explainer_chain],
    input_variables=['question'],
    output_variables=['answer'],
    verbose=True
)

In [13]:
response = overall_chain({"question": "What are the top 5 albums by Jimi hendrix?"})
print(response['answer'])

/var/folders/cn/1py0zvm95nx9kp_1_zqwycrm0000gn/T/ipykernel_12917/1142704740.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = overall_chain({"question": "What are the top 5 albums by Jimi hendrix?"})




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Using this code:

```python
def solution():
    """What are the top 5 albums by Jimi hendrix?"""
    search_results = sp.search(q='Jimi Hendrix', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    albums = sp.artist_albums(uri)
    top_albums = sorted(albums['items'], key=lambda x: x.get('popularity'), reverse=True)[:5]
    relevant_data = []
    for album in top_albums:
        relevant_data.append({
            'album_name': album.get('name'),
            'album_uri': album.get('uri'),
            'artist_uri': uri,
            'popularity_score': album.get('popularity')
        })
    return relevant_data
```

We got the following output from the Spotify API:

```json
The code provided is a Python script that uses the Spotify API to retrieve information about the top 5 albums by Jimi Hendrix. 

Here's a breakdown of the code:
1. The script imports the ne

In [15]:
import gradio as gr


def spotify_query(question):
    response = overall_chain({"question": question})
    return response['answer']

demo = gr.Interface(
    fn=spotify_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your Spotify query here..."),
    outputs="text",
    title="SpotSearch AI",
    description="Ask questions about artists, albums, songs, and more on Spotify!",
    examples=[
        "What are the top 5 albums by Jimi Hendrix?",
        "What are the most danceable songs from Kendrick Lamar?",
        "What are the first three songs on The Dark Side Of The Moon?",
    ]
)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Using this code:

```python
def solution():
    """Give me 10 songs similar to The Great Gig in the Sky by Pink Floyd."""
    search_results = sp.search(q='Pink Floyd', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    tracks = sp.artist_top_tracks(uri)
    track_uris = [track.get('uri') for track in tracks['tracks']]
    audio_details = sp.audio_features(track_uris)
    similar_songs = []
    for i, details in enumerate(audio_details):
        if details['tempo'] == 'similar':
            track = tracks['tracks'][i]
            similar_songs.append({
                'song_name': track.get('name'),
                'artist_name': track.get('artists')[0].get('name'),
                'album_name': track.get('album').get('name'),
                'song_uri': track.get('uri'),
                'artist_uri': track.get('artists')[0].get('uri'),
                'alb